# EXTRACT EVENTS FLOODLIST

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import datetime
import re
import io
import csv
import os

In [2]:
key = '' #Key for Floodlist API
url = "http://floodlist.com/api?key=%s" % key
data = requests.get(url).json()

In [3]:
events = []

for item in data:
    
    if (item['start_date'] >= '2018-08-01' and item['start_date']<='2019-04-01'):
        
        countries = []
        for country in item['countries']:
            countries.append(country['country'])
            
        types = [t for t in item['flood_event_types']]
        
        locations = [locat for locat in item['locations']]
        
        magnitudes = []
        
        for mag in item['flood_magnitude']:
            if (mag['title'] == 'Rainfall level'):
                magnitudes.append(mag)
        
        links = []
        
        for link in item['related_links']:
            links.append(link['url'])
                
        
        damages = [dam for dam in item['flood_damage']]
        
        dictionary = {}
        
        dictionary['start_date'] = item['start_date']
        dictionary['end_date'] = item['end_date']
        dictionary['countries'] = ', '.join(countries)
        dictionary['types'] = ', '.join(types)
        dictionary['locations'] = locations
        dictionary['magnitudes'] = magnitudes
        dictionary['damages'] = damages
        dictionary['id_floodlist'] = ', '.join(links)
        
        events.append(dictionary)

In [4]:
df = pd.DataFrame.from_records(events)

In [5]:
df.head()

,start_date,end_date,countries,types,locations,magnitudes,damages,id_floodlist
0,2019-04-01,,Colombia,"Flash flood, Landslide, River flood","[{'title': 'Vigía del Fuerte, Antioquia', 'lon...","[{'title': 'Rainfall level', 'amount': '92', '...","[{'title': 'Fatalities', 'amount': '10', 'unit...",http://floodlist.com/america/colombia-floods-l...
1,2019-04-01,,Ghana,Flash flood,"[{'title': 'Accra', 'longitude': '-0.186964399...","[{'title': 'Rainfall level', 'amount': '214.9'...","[{'title': 'Fatalities', 'amount': '5', 'unit'...",http://floodlist.com/africa/ghana-floods-accra...
2,2019-03-24,,Bolivia,River flood,"[{'title': 'Charagua', 'longitude': '-63.19732...","[{'title': 'Rainfall level', 'amount': '49', '...","[{'title': 'Affected', 'amount': '8,500', 'uni...",http://floodlist.com/america/floods-paraguay-p...
3,2019-03-19,,Peru,"Flash flood, Landslide, River flood","[{'title': 'Pomabamba', 'longitude': '-77.4615...","[{'title': 'Rainfall level', 'amount': '44.8',...","[{'title': 'Evacuated', 'amount': '40', 'unit'...",http://floodlist.com/america/floods-paraguay-p...
4,2019-03-21,,Ecuador,"Flash flood, River flood","[{'title': 'Pueblo Viejo', 'longitude': '-79.5...","[{'title': 'Rainfall level', 'amount': '73.1',...","[{'title': 'Affected', 'amount': '8,000', 'uni...",http://floodlist.com/america/floods-paraguay-p...


In [6]:
df.index = df.pop('start_date')
df.rename(columns={'countries': 'Country'}, inplace=True)
df.head()

,end_date,Country,types,locations,magnitudes,damages,id_floodlist
start_date,,,,,,,
2019-04-01,,Colombia,"Flash flood, Landslide, River flood","[{'title': 'Vigía del Fuerte, Antioquia', 'lon...","[{'title': 'Rainfall level', 'amount': '92', '...","[{'title': 'Fatalities', 'amount': '10', 'unit...",http://floodlist.com/america/colombia-floods-l...
2019-04-01,,Ghana,Flash flood,"[{'title': 'Accra', 'longitude': '-0.186964399...","[{'title': 'Rainfall level', 'amount': '214.9'...","[{'title': 'Fatalities', 'amount': '5', 'unit'...",http://floodlist.com/africa/ghana-floods-accra...
2019-03-24,,Bolivia,River flood,"[{'title': 'Charagua', 'longitude': '-63.19732...","[{'title': 'Rainfall level', 'amount': '49', '...","[{'title': 'Affected', 'amount': '8,500', 'uni...",http://floodlist.com/america/floods-paraguay-p...
2019-03-19,,Peru,"Flash flood, Landslide, River flood","[{'title': 'Pomabamba', 'longitude': '-77.4615...","[{'title': 'Rainfall level', 'amount': '44.8',...","[{'title': 'Evacuated', 'amount': '40', 'unit'...",http://floodlist.com/america/floods-paraguay-p...
2019-03-21,,Ecuador,"Flash flood, River flood","[{'title': 'Pueblo Viejo', 'longitude': '-79.5...","[{'title': 'Rainfall level', 'amount': '73.1',...","[{'title': 'Affected', 'amount': '8,000', 'uni...",http://floodlist.com/america/floods-paraguay-p...


In [7]:
df['in_floodlist'] = 1 #Add identifier for future merging

In [8]:
df.rename(columns={'Country': 'countries', 'magnitudes': 'rainfall_level', 'types': 'type'}, inplace=True)
df

,end_date,countries,type,locations,rainfall_level,damages,id_floodlist,in_floodlist
start_date,,,,,,,,
2019-04-01,,Colombia,"Flash flood, Landslide, River flood","[{'title': 'Vigía del Fuerte, Antioquia', 'lon...","[{'title': 'Rainfall level', 'amount': '92', '...","[{'title': 'Fatalities', 'amount': '10', 'unit...",http://floodlist.com/america/colombia-floods-l...,1
2019-04-01,,Ghana,Flash flood,"[{'title': 'Accra', 'longitude': '-0.186964399...","[{'title': 'Rainfall level', 'amount': '214.9'...","[{'title': 'Fatalities', 'amount': '5', 'unit'...",http://floodlist.com/africa/ghana-floods-accra...,1
2019-03-24,,Bolivia,River flood,"[{'title': 'Charagua', 'longitude': '-63.19732...","[{'title': 'Rainfall level', 'amount': '49', '...","[{'title': 'Affected', 'amount': '8,500', 'uni...",http://floodlist.com/america/floods-paraguay-p...,1
2019-03-19,,Peru,"Flash flood, Landslide, River flood","[{'title': 'Pomabamba', 'longitude': '-77.4615...","[{'title': 'Rainfall level', 'amount': '44.8',...","[{'title': 'Evacuated', 'amount': '40', 'unit'...",http://floodlist.com/america/floods-paraguay-p...,1
2019-03-21,,Ecuador,"Flash flood, River flood","[{'title': 'Pueblo Viejo', 'longitude': '-79.5...","[{'title': 'Rainfall level', 'amount': '73.1',...","[{'title': 'Affected', 'amount': '8,000', 'uni...",http://floodlist.com/america/floods-paraguay-p...,1
...,...,...,...,...,...,...,...,...
2018-08-11,2018-08-14,United States,"Urban flood, Flash flood","[{'title': 'Brick, NJ', 'longitude': '-74.1371...","[{'title': 'Rainfall level', 'amount': '198.88...","[{'title': 'Evacuated', 'amount': '60', 'unit'...",http://floodlist.com/america/usa/floods-newyor...,1
2018-08-15,2018-08-16,India,Flash flood,"[{'title': 'Sultangarh Water Fall', 'longitude...","[{'title': 'Rainfall level', 'amount': ' 70.5'...","[{'title': 'Fatalities', 'amount': '11', 'unit...",http://floodlist.com/asia/india-madhya-pradesh...,1
2018-08-11,2018-08-12,Colombia,Flash flood,"[{'title': 'Mocoa', 'longitude': '-76.65105499...",[],"[{'title': 'Evacuated', 'amount': '30,000', 'u...",http://floodlist.com/america/colombia-floods-p...,1


In [9]:
outfile = './data/databases/floodlist.csv'
df.to_csv(outfile, sep=',')